# Introduction

#market-risk, #var, #variance-covariance, #sbm, #frtb, #cva-frtb, #simm

Run this notebook to create an analytical application for the SBM charge. The input data will be stored in-memory and Atoti will perform the computation "on-the-fly". Users can filter and drill down to explore your data.

You will find below how the SBM logic can be described in python and injected into Atoti. We'll narrow down the use case to Equity Delta charge, the other aggregation chains can be added in a similar manner.

Sensitivity-based mathod is one of the parametric market risk methodologies. It can be used to compute a VaR-like metric from sensitivities using pre-calibrated risk weights and correlations, through a sequence of nested variance covariance formulae. Due to its multiple benefits (see [2]), the method is widely used both internally by orgazations to manage market risk, as well as by Regulators for capital requirements purposes (FRTB) as well as margining (SIMM). 

The notebook assumes that the reader is familiar with the sensitivity-based method and terminology.

## Input data

Input data is sourced in a CRIF-like format: https://www.isda.org/a/owEDE/risk-data-standards-v1-36-public.pdf

## References 

- [1]: Consolidated Basel Framework Chapter MAR21: https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101. 
- [2]: ISDA SIMM(TM): From Principles to Model Specification https://www.isda.org/a/vAiDE/simm-from-principles-to-model-specification-4-mar-2016-v4-public.pdf

# Imports

In [1]:
from IPython.display import display, Markdown, Latex, Image
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_colwidth", 300)
pd.options.display.float_format = '{:,.4f}'.format


# Atoti

In [2]:
import atoti as av
from atoti.config import create_config
session = av.create_session(
    config='./configuration.yaml', port='53972', sampling_mode=av.sampling.FULL)

Welcome to Atoti 0.3.1!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


# Url

In [3]:
display(Markdown('[' + session.url+ ']('+ session.url+')'))

[http://localhost:53972](http://localhost:53972)

# Input data files

These are the csv-files that we need to illustrate the Equity Delta aggregation:

- "smaller_data.csv" - sensitivities in a CRIF-like format
- "eq_delta_gamma.csv" - cross-bucket correlations, set for each pair of buckets
- "eq_delta_rho.csv" - risk factor correlations, set per bucket, i.e. all pairs of risk factors in a given bucket have the same correlation
- "eq_delta_rw.csv" - risk weights, set per bucket and per risk factor type (stored in Label2 crif field)

The example data files are stored in a cloud and I'm dowloading them to the working directory.

In [4]:
# this cell will download example data files from a url to the working directory
import sys  
sys.path.append('../../utils')

import notebook_utils
cwd = os.getcwd() # to be removed

notebook_utils.download_source(cwd, 'https://data.atoti.io/notebooks/sbm/smaller_data.csv', "smaller_data.csv")
notebook_utils.download_source(cwd, 'https://data.atoti.io/notebooks/sbm/bigger_data.csv', "bigger_data.csv")

notebook_utils.download_source(cwd, 'https://data.atoti.io/notebooks/sbm/parameters/eq_delta_gamma.csv', "eq_delta_gamma.csv")
notebook_utils.download_source(cwd, 'https://data.atoti.io/notebooks/sbm/parameters/eq_delta_rho.csv', "eq_delta_rho.csv")
notebook_utils.download_source(cwd, 'https://data.atoti.io/notebooks/sbm/parameters/eq_delta_rw.csv', "eq_delta_rw.csv")


smaller_data.csv already downloaded.
bigger_data.csv already downloaded.
eq_delta_gamma.csv already downloaded.
eq_delta_rho.csv already downloaded.
eq_delta_rw.csv already downloaded.


# Sensitivities datastore

In [5]:
# in this example, the data is initially read into a pandas dataframe, 
# which is subsequently used to create the datastore.
crif = pd.read_csv("smaller_data.csv")
crif = crif.append(pd.read_csv("bigger_data.csv"), ignore_index = True)
crif.head(5)

,TradeID,RiskType,Qualifier,Label2,AmountUSD,Bucket,PortfolioID
0,0,Risk_Equity,Wilmar International,REPO,"-10,332.0900",1,Smaller_Portfolio
1,1,Risk_Equity,Wilmar International,SPOT,"-3,641,606.4500",1,Smaller_Portfolio
2,2,Risk_Equity,China Minmetals,REPO,"-2,337.9000",3,Smaller_Portfolio
3,3,Risk_Equity,China Minmetals,SPOT,"-10,549.5400",3,Smaller_Portfolio
4,4,Risk_Equity,China Life Insurance,REPO,"-7,874.9400",4,Smaller_Portfolio


In [6]:
risks_store = session.read_pandas(crif,
                                  keys=['TradeID', 'PortfolioID','RiskType', 'Qualifier', 'Label2'],
                                  store_name="Risks",
                                  types={'Bucket': av.types.STRING})

cube = session.create_cube(risks_store)
lvl = cube.levels
m = cube.measures
h = cube.hierarchies

In [7]:
# creating a comparator to sort buckets as numbers:
# NOT WORKING AT THE MOMENT
# lvl["Bucket"].comparator = av.comparator.first_members([str(i) for i in range(1,13)])

At this point, a cube has been created, and we can start browsing the sensitivities:

In [9]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

Run this command to create a new data vizualisation:

In [10]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

# Risk weights datastore

In [11]:
# the risk weights data store is created from a csv:
eq_delta_risk_weights_store = session.read_csv(
    "data/parameters/eq_delta_rw.csv",
    keys=["Bucket", "Label2"],
    types={"Bucket": av.types.STRING},
    store_name="RiskWeights"
)

# risks store is joined with the risk weights store
risks_store.join(eq_delta_risk_weights_store)

In [12]:
eq_delta_risk_weights_store.head(5)

,,RW
Bucket,Label2,
1,SPOT,0.5500
2,SPOT,0.6000
3,SPOT,0.4500
4,SPOT,0.5500
5,SPOT,0.3000


# Weighted sensitivities

In this section, we'll create a measure visualizing the weighted sensitivities defined in
[MAR21.4](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_4):

$$WS_k=RW_k \cdot s_k$$

As the risk weights are defined for each Bucket and Label2, this is the level where sensitivities need to be multiplied by the risk weight.


In [13]:
# The input sensitivities are multplied by risk weight for each Bucket and Label2,
# and then summed up to obtain weighted sensitivities
m['WS'] = av.agg.sum(m['AmountUSD.SUM']*m['RW.VALUE'], scope=av.scope.origin("Bucket","Label2"))

# Bucket-level aggregation

The weighted sensitivities are rolled up into the bucket-level charges, using a variance-covariance formula that can be found in the [MAR21.4](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_4):

$$K_{b} =\sqrt{max \left( 0, \sum _{k\in b} WS_{k}^{2} +\sum _{k\in b}\sum
  _{l\in b, l\neq k}\rho_{kl}\cdot WS_k \cdot WS_l\right)}$$
  
In this section we'll create measures visualizing bucket-level charges. 

The formula for bucket-level charges is a type of variance-covariance aggregation. We will provide two methods to compute it:

1. Method 1: materializing both risk factors in a pair and looping over all the pais of the risk factors
2. Method 2: more computationally efficient: leveraging the fact, that some of the risk factors use the same correlation values, we provide a more computationally efficient calculation

## Risk factor correlations

The parameter $\rho_{kl}$ denotes correlation between two risk factors $k$ and $l$ in a pair of risk factors. The rules defining the equity delta correlations are set in [MAR21.78](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_78).

The rules can be summarised for each pair of risk factors as follows:

- Case 1: same name, different type: a single value -> 0.999
- Case 2: different name, same type: a single value depending on bucket, for example, 0.15
- Case 3: different name, different type: value depending on risk factor multiplier by 0.999, for example, 0.15 x 0.999

For example,


| risk factors | name1-spot | name1-repo | name2-spot | name2-repo |
|------------|-------------|-------------|-------------|-------------|
| name1-spot | 1 |  |  |  |
| name1-repo | same_name_diff_type | 1 |  |  |
| name2-spot | rho_by_name | rho_by_name x type_multiplier | 1 |  |
| name2-repo | rho_by_name x type_multiplier | rho_by_name x type_multiplier | same_name_diff_type | 1 |

In [14]:
# Equity delta risk factor is defined as a combination of fields - "Qualifier" and "Label2", i.e. equity name and risk factor type.
# Creating variables:
same_risk_factor = 1.0
same_name_diff_type = 0.999
diff_type_multiplier = 0.999

In [15]:
# Creating a datastore holding correlations defined per bucket ([MAR21.78](2)):
eq_delta_rho = session.read_csv(
    "eq_delta_rho.csv", keys=["Bucket"], types={"Bucket": av.types.STRING}, 
    store_name="RiskFactorCorrelations"
)
risks_store.join(eq_delta_rho)
eq_delta_rho.head(5)

,names_correlation
Bucket,
1,0.1500
2,0.1500
3,0.1500
4,0.1500
5,0.2500


## Method 1: materializing risk factor pairs

In [16]:
# Important:
# We join by POrtfolioID - hence  
# the combinations of risk factors are restricted to those that defined below PortfolioID

In [17]:
# Now, for each risk factor - "Qualifier" + "Label2" - there's a list of all risk factors in portfolio and bucket.
# THe hierarchies representing risk factors: "Other Qualifier", "Other Label2".
other_risk_factor_store = session.read_pandas(crif[['PortfolioID', 'Bucket', 'Qualifier', 'Label2']].rename(columns={'Qualifier': 'Other Qualifier', 'Label2': 'Other Label2'}), keys=[
                                              'PortfolioID', 'Bucket', 'Other Qualifier', 'Other Label2'], types={'Bucket': av.types.STRING}, store_name='OtherRiskFactor')
risks_store.join(other_risk_factor_store, mapping={
                 'PortfolioID': 'PortfolioID', 'Bucket': 'Bucket'})

In [18]:
# Setting up a measure returning correlation for any pair of 
# risk factors represented by hierarchies ("Qualifier", "Label2") and ("Other Qualifier", "Other Label2")
m['rho_kl'] = av.where(lvl['Qualifier'] == lvl['Other Qualifier'], 
                       av.where(lvl['Label2'] == lvl['Other Label2'], same_risk_factor, same_name_diff_type),
                       av.where(lvl['Label2'] == lvl['Other Label2'], m['names_correlation.VALUE'],
                                m['names_correlation.VALUE'] * diff_type_multiplier))

In [19]:
m['WSk'] = m['WS']
m['WSl'] = av.at(m['WS'], {
                        lvl['Qualifier']: lvl['Other Qualifier'],
                        lvl['Label2']: lvl['Other Label2']})

# Please note, that for the bucket 11, bucket-level charge is defined as sum of absoluted 
# weighted sensitivities by risk factor [MAR21.79](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_79)

m['Kb'] = av.agg.stop(av.where(lvl['Bucket'] == '11',
                                av.agg.sum(av.abs(m['WSk']), av.scope.origin('Qualifier','Label2')),
                                av.sqrt(av.max(0, av.agg.sum(m['WSk']
                                                             * m['WSl'] * m['rho_kl'], scope = av.scope.origin('Qualifier','Other Qualifier','Label2','Other Label2'))))), at = [lvl['Bucket']])

In [20]:
m['temp'] = m['WSk']    * m['WSl'] * 0.999

In [21]:
m['sq root of temp'] = av.sqrt(m['temp'])

In [22]:
cube.query(m['AmountUSD.SUM'], m['WS'], m['Kb'], levels = [lvl['Bucket']], condition = lvl['PortfolioID']=="Smaller_Portfolio")

,AmountUSD.SUM,WS,Kb
Bucket,,,
1,"-3,651,938.5400","-2,002,940.3740","2,002,940.3172"
10,"-5,652,387.6950","-2,810,353.8262","1,819,315.9010"
11,"9,197,909.9830","6,418,348.7082","6,418,348.7082"
12,"-1,913,082.0100","-286,992.4351","286,992.4354"
3,"-12,887.4400","-4,757.8136","4,757.8031"
4,"-1,030,083.3000","-557,071.4497","758,465.4868"
5,"1,443,720.0700","429,349.4284","456,861.0617"
7,"239,581.2030","91,493.4106","932,682.8928"
8,"-10,423,086.7700","-5,177,444.7259","3,937,788.9195"


## Method 2: optimized Kb calculation

Since many of the risk pairs share the same correlation value, it is possible to optimize the variance-covariance aggregation. The efficiency of this calculation is critical when data cardinality along risk factor is high.

We decompose the formula into the three components:
- contribution of pairs with both risk factors being spot - same correlation $\rho_{names}$ set per bucket,
- contribution of pairs with both risk factors being repo - same correlation $\rho_{names}$ set per bucket,
- contribution of pairs where one risk factor is spot, another one is repo - same correlation $\rho_{names} \cdot 0.999$. We'd need to account that for some of the pairs where one risk factor is spot, one is repo some the equity names will match and need to be correlated at 0.999


Let's start with the pairs where both risk factors are either spot or repo. Their contribution can be rewritten:

$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l = \sum_k WS_k^2 + \rho_{kl} \cdot \left(\left( \sum_k WS_k \right)^2 - \sum_k WS_k^2  \right) = (1-\rho_{kl}) \cdot \sum_k WS_k^2 + \rho_{kl} \cdot \left( \sum_k WS_k \right)^2  $$

In [23]:
# This measure will display sum by k of WS_k squared:
m['sum squares'] = av.agg.square_sum(m['WS'], scope = av.scope.origin('Qualifier', 'Label2'))
m['reduced formula'] = (1-m['names_correlation.VALUE'])*m['sum squares'] + m['names_correlation.VALUE']* av.pow(m['WS'],2.0)

# contribution of pairs having only spot risk factors:
m['spot contribution'] = av.filter(m['reduced formula'] , lvl['Label2']=="SPOT")
# contribution of pairs having only repo risk factors:
m['repo contribution'] = av.filter(m['reduced formula'] , lvl['Label2']=="REPO")

The contribution of the risk factors where one risk factor belongs to "SPOT" and the other belongs to "REPO" can be rewritten:
$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l =  \vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}} \cdot \rho_{names} \cdot 0.999 + 0.999 \cdot (1 - \rho_{names}) \sum_{n\in names}{WS_n^{repo} \cdot WS_n^{spot}}$$

where:
- J - is a matrix of ones,
- first term in the above formula performs aggregation of all sensitivities, as if they all are correlated at $\rho_{names}$,
- the second term is to correct the first term and to account for the fact that risk factors, where spot and repo risk factors have the same equity name, must be correlated at 0.999.


In [24]:
# filtering for repo and spot risk factors:
m['WS_spot'] = av.filter(m['WSk'], lvl['Label2']=="SPOT")
m['WS_repo'] = av.filter(m['WSk'], lvl['Label2']=="REPO")

The measure `sum WS_repo J WS_spot` will display $\vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}} $:

In [25]:
# m['WS_spot_vector'] = av.parent_value(av.agg._vector(m['WS_spot'], av.scope.origin('Qualifier')), on_hierarchies=[h['Bucket'],h['Qualifier']])
# m["sum WS_repo J WS_spot"] = av.agg.sum(av.array.sum(m['WS_spot_vector']*m['WS_repo']), scope = av.scope.origin('Qualifier', 'Bucket'))

In [ ]:
# cube.query(m['WS_repo J WS_spot'], m['repo scalar x spot vector'], m['array sum'], levels = [lvl['Bucket'], lvl['Qualifier']])

In [44]:
# Collect WS of the spot risk factors in a vector and show against every qualifier in a bucket:
weights_vector = av.agg._vector(m['WS_spot'], av.scope.origin('Bucket', 'Qualifier'))
m['spot vector'] = av.parent_value(weights_vector, on_hierarchies = [ h['Qualifier']])

# Multiply vector WS of the spot risk factors by WS of each repo risk factor sum them up
m['repo scalar x spot vector'] = m['WS_repo'] * m['spot vector']
m['WS_repo J WS_spot'] = av.agg.sum(av.array.sum(m['repo scalar x spot vector']), scope = av.scope.origin('Bucket', 'Qualifier'))

In [50]:
m['array sum'] = av.array.sum(m['repo scalar x spot vector'])

In [52]:
m['sum of array sum'] = av.agg.sum(m['array sum'], scope = av.scope.origin('Bucket', 'Qualifier'))

# PROBLEM

We want to have a measure, which will be a simple of array sum, but for some reason it shows empty/None!

In [53]:
cube.query(m['sum of array sum'])

,sum of array sum


In [57]:
av.__version__

'0.3.1'

In [59]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

In [ ]:
######### Below formula will multiply by null - might be a bad practice??

In [33]:
# cross product of weighted sensitivities for the names, having both spot and repo sensitivities:
m['sum WS repo and spot'] = av.agg.sum(m['WS_spot']*m['WS_repo'], scope = av.scope.origin('Qualifier'))

In [34]:
# final contribution of the pairs where one risk factor is spot, one is repo
m["cross repo spot contribution"] = m['names_correlation.VALUE']*m["sum WS_repo J WS_spot"]+0.999*(1- m['names_correlation.VALUE'])* m['sum WS repo and spot']

In [45]:
cube.query( m['cross repo spot contribution'], m['sum WS repo and spot'], m["sum WS_repo J WS_spot"], levels = [lvl['Bucket']], condition = lvl['PortfolioID']=="Smaller_Portfolio")

,cross repo spot contribution,sum WS repo and spot,sum WS_repo J WS_spot


In [ ]:
# Hence the formula for Kb can be repre

In [35]:
m['K_b alternative'] = av.sqrt(av.max(0, m['spot contribution'] + m['repo contribution'] + 2*m["cross repo spot contribution"]))

In [39]:
cube.query(m['Kb'], m['K_b alternative'], m['spot contribution'], m['repo contribution'],levels = [lvl['PortfolioID'], lvl['Bucket']], condition = lvl['PortfolioID']=="Smaller_Portfolio")

Kb  K_b alternative  \
PortfolioID       Bucket                                   
Smaller_Portfolio 1      2,002,940.3172   2,002,883.5483   
                  10     1,819,315.9010   1,819,191.3827   
                  11     6,418,348.7082   3,409,633.5532   
                  12       286,992.4354     286,992.7395   
                  3          4,757.8031       4,747.3047   
                  4        758,465.4868     758,430.0265   
                  5        456,861.0617     456,846.2609   
                  7        932,682.8928     932,701.1300   
                  8      3,937,788.9195   3,937,638.7286   
                  9      2,234,809.9610   2,234,768.0819   

                               spot contribution  repo contribution  
PortfolioID       Bucket                                             
Smaller_Portfolio 1       4,011,542,504,846.1855         3,229.2505  
                  10      3,309,457,252,220.8950        34,792.3173  
                  11     11,625,600,950,839.9102        16,133.3217  
                  12         82,364,832,525.7148             0.0926  
                  3              22,536,790.8278           110.6820  
                  4         575,216,099,754.0173         5,301.2157  
                  5         208,708,503,926.5409         2,188.4072  
                  7         869,931,393,755.0397         4,196.0403  
                  8      15,504,998,703,492.8867        53,668.7411  
                  9       4,994,188,329,084.8691        50,755.9984

In [44]:
cube.query(m['Kb'], m['K_b alternative'], levels = [lvl['PortfolioID'], lvl['Bucket']], condition = lvl['PortfolioID']=="Smaller_Portfolio")

Kb  K_b alternative
PortfolioID       Bucket                                
Smaller_Portfolio 1      2,002,940.3172   2,002,883.5483
                  10     1,819,315.9010   1,819,191.3827
                  11     6,418,348.7082   3,409,633.5532
                  12       286,992.4354     286,992.7395
                  3          4,757.8031       4,747.3047
                  4        758,465.4868     758,430.0265
                  5        456,861.0617     456,846.2609
                  7        932,682.8928     932,701.1300
                  8      3,937,788.9195   3,937,638.7286
                  9      2,234,809.9610   2,234,768.0819

# Cross-bucket aggregation

## Bucket correlations

In [56]:
eq_delta_buckets_correlations = session.read_csv(
    "eq_delta_gamma.csv", keys=["Bucket", "Other Bucket"], types = {"Bucket": av.types.STRING, "Other Bucket": av.types.STRING}, store_name="eq_delta_corr_outer"
)
risks_store.join(eq_delta_buckets_correlations)

## Aggregating across buckets

In [61]:
# 21.4(5)(a):
m['WSb'] = m['WS']
m['WSc'] = av.at(m['WS'], {lvl['Bucket']: lvl['Other Bucket']})
m['sum Kb2 + sum sum WSb WSc gamma'] = av.agg.square_sum(m['Kb'], av.scope.origin('Bucket')) + av.agg.sum(
    m['WSb'] * m['WSc']*m['gamma.VALUE'], scope = av.scope.origin('Bucket','Other Bucket'))

# 21.4(5)(b):
m['Sb'] = av.max(av.min(m['WS'], m['Kb']), -1.0 * m['Kb'])
m['Sc'] = av.at(m['Sb'], {lvl['Bucket']: lvl['Other Bucket']})
m['sum Kb2 + sum sum Sb Sc gamma'] = av.agg.square_sum(m['Kb'], av.scope.origin(lvl['Bucket'])) + av.agg.sum(
    m['Sb'] * m['Sc']*m['gamma.VALUE'], scope = av.scope.origin('Bucket','Other Bucket'))

m['Delta Margin'] = av.where(m['sum Kb2 + sum sum WSb WSc gamma'] > 0,
                             av.sqrt(m['sum Kb2 + sum sum WSb WSc gamma']),
                             av.sqrt(m['sum Kb2 + sum sum Sb Sc gamma']))

In [63]:
cube.query(m['Delta Margin'], condition = lvl['PortfolioID']=="Smaller_Portfolio", timeout=90)

,Delta Margin
0,"8,951,153.7749"


In [ ]:
# Optimized calculation

# 21.4(5)(a):
m['sum Kb2 + sum sum WSb WSc gamma alternative'] = av.agg.square_sum(m['K_b alternative'], av.scope.origin([lvl['Bucket']])) + av.agg.sum(
    m['WS_b'] * m['WS_c']*m['gamma.VALUE'], av.scope.origin([lvl['Bucket'], lvl['Other Bucket']]))

# 21.4(5)(b):
m['S_b alternative'] = av.max(av.min(m['WS_k'], m['K_b alternative']), -1.0 * m['K_b alternative'])
m['S_c alternative'] = av.at(m['S_b alternative'], {lvl['Bucket']: lvl['Other Bucket']})
m['sum Kb2 + sum sum Sb Sc gamma alternative'] = av.agg.square_sum(m['K_b alternative'], av.scope.origin([lvl['Bucket']])) + av.agg.sum(
    m['S_b alternative'] * m['S_c alternative']*m['gamma.VALUE'], av.scope.origin([lvl['Bucket'], lvl['Other Bucket']]))

m['Delta Margin alternative'] = av.where(m['sum Kb2 + sum sum WSb WSc gamma alternative'] > 0,
                             av.sqrt(m['sum Kb2 + sum sum WSb WSc gamma alternative']),
                             av.sqrt(m['sum Kb2 + sum sum Sb Sc gamma alternative']))

In [ ]:
cube.query(m['Delta Margin alternative'], levels = [lvl['PortfolioID']], timeout=90)

# Timeit

In [ ]:
risks_store.load_pandas(crif)

In [ ]:
%%timeit
cube.query(m['Delta Margin'], condition = lvl['PortfolioID']=="Smaller_Portfolio", timeout=90)

In [ ]:
%%timeit
cube.query(m['Delta Margin'], condition = lvl['PortfolioID']=="Bigger_Portfolio", timeout=90)

In [ ]:
%%timeit
cube.query(m['Delta Margin alternative'], condition = lvl['PortfolioID']=="Smaller_Portfolio")

In [ ]:
%%timeit
cube.query(m['Delta Margin alternative'], condition = lvl['PortfolioID']=="Bigger_Portfolio")